**Содержание**<a id='toc0_'></a>    
- [Агрегация, механизм внимания. Трансформер и self-attention](#toc1_)    
  - [Агрегация (pooling)](#toc1_1_)    
    - [Пример агрегации (пуллинга)](#toc1_1_1_)    
      - [Проблемы пуллинга:](#toc1_1_1_1_)    
  - [Идея умного пуллинга - механизм внимания](#toc1_2_)    
  - [Механизм внимания как набор компонентов](#toc1_3_)    
  - [Популярные архитектуры механизма внимания](#toc1_4_)    
  - [Сравнение текстов с механизмом внимания](#toc1_5_)    
- [Трансформер и self-attention](#toc2_)    
  - [Механизм внутреннего внимания (self-attention, intra attention)](#toc2_1_)    
    - [Использование раздельных проекций для запросов, ключей и значений](#toc2_1_1_)    
    - [Недостатки механизма внимания](#toc2_1_2_)    
    - [Повышение качества механизма внутреннего внимания. Multihead self-attention](#toc2_1_3_)    
  - [Управление зависимостями](#toc2_2_)    
- [Трансформер](#toc3_)    
  - [Позиционное кодирование](#toc3_1_)    
- [Теоретические вопросы: Модель языка и трансформеры](#toc4_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[Агрегация, механизм внимания. Трансформер и self-attention](#toc0_)

История о том, как получить один вектор, представляющий целое предложение или даже текст. То есть — как **агрегировать глобальный контекст**.

## <a id='toc1_1_'></a>[Агрегация (pooling)](#toc0_)
Вход:

$$Length \times EmbeddingSize$$

Выход:

$$NewLength \times EmbeddingSize \text{ или } EmbeddingSize$$

Цель:

- расширение контекста с потерей пространственной информации
- учет глобального контекста
  
Агрегация (пуллинг) работают подобно свертке
- проход по входу окном и выбор:
  - среднего
  - максимума
- в отличие от свертки, применяется к каждому каналу (столбцы) независимо
- никаких параметров кроме самого вида преобразования (среднее или максимум нету)
- повторяется до тех пор пока не останется 1 вектор 
- в итоге учитывается весь контекст документа

### <a id='toc1_1_1_'></a>[Пример агрегации (пуллинга)](#toc0_)
Операция агрегации (pooling) принимает на вход матрицу $Input \in \mathbb{R} ^ {InLen \times EmbSize}$, где $InLen$ - количество строк, соответствующее длине входной последовательности, а $EmbSize$ - количество столбцов, соответствующее количеству признаков для каждого элемента.

У агрегации два основных гиперпараметра:

$k$ - размер окна (как для свёртки)
$f(x): \mathbb{R}^k \rightarrow \mathbb{R}$ - функция агрегации, как правило это нахождение среднего значения `avg(x)` или нахождение максимального значения `max(x)`

Результат агрегации - новая матрица $Output \in \mathbb{R} ^ {OutLen \times EmbSize}$, где $OutLen = InLen - k + 1$ - длина выходной последовательности.

Тогда операцию агрегации можно записать следующей формулой: 
$$Output[OutPos, Ch] = f(Input[OutPos:OutPos + k, Ch])$$

В этой формуле:

$OutPos$ - номер строки в результирующей матрице (то есть порядковый номер элемента в выходной последовательности), 
$Ch$ - номер столбца (то есть номер признака), 
$Input[OutPos:OutPos + k, Ch] \in \mathbb{R}^k$ - вектор из $k$ значений признака $Ch$, соответствующих входным элементам последовательности начиная с позиции $OutPos$ до позиции $OutPos + k - 1$ включительно (то есть как `slice` в `NumPy`)

Примените операцию max-пулинга с ядром $k=2$ к матрице $(InLen=3, EmbSize=3)$

$$Input = \left( \begin{matrix} 1 & 0 & 2 \\ 0 & 1 & 3 \\ 1 & 3 & 0 \end{matrix} \right) \in \mathbb{R} ^ {InLen \times EmbSize}$$

Шаг скользящего окна (`stride` в `PyTorch`) считаем равным 1.

**A:**

$$Input = \left( \begin{matrix} 1 & 0 & 2 \\ 0 & 1 & 3 \\ 1 & 3 & 0 \end{matrix} \right) \rightarrow 
\left( \begin{matrix} 1 & 1 & 3 \\ 1 & 3 & 3 \end{matrix} \right) \rightarrow 
\left( \begin{matrix} 1 & 3 & 3 \end{matrix} \right) $$


In [1]:
import torch
from torch import nn

data = [[[1.0,0.0,2.0],
         [0.0,1.0,3.0],
         [1.0,3.0,0.0]]]            

x_data = torch.tensor(data)                 # InLen X EmbSize
m = nn.MaxPool1d(kernel_size=2, stride=1)

m(x_data.transpose(1, 2)).transpose(1, 2)

tensor([[[1., 1., 3.],
         [1., 3., 3.]]])

- Такой пуллинг является "**мягким**" в смысле постепенных, но можно применять и **глобальный пуллинг** (ширина окна == размеру входа) - за 1 шаг получаем сразу вектор, т.е. простое среднее/максимум по всем входам
- как правило глобальный пуллинг используют ближе к выходным слоям нейросети
- после него обычно идет полносвязный слой, т.е. (мульти)линейный классификатор (например, для задачи классификации)

#### <a id='toc1_1_1_1_'></a>[Проблемы пуллинга:](#toc0_)
- каждый канал агрегируется независимо
- амплитуда агрегированного значения отвечает и за значимость и за полезную информацию одновременно
- это слишком слабое преобразование для многих задач (особенно глобальный вариант)

**Примените операцию глобального avg-пулинга к матрице.**
 
$$avg(Input) = \frac{1}{k} \sum_{i=0}^{k-1} Input[i]$$

$$Input = \left( \begin{matrix} 1 & 0 & 2 \\ 0 & 1 & 3 \\ 1 & 3 & 0 \\ 0 & 0 & 0\end{matrix} \right) \in \mathbb{R} ^ {InLen \times EmbSize} \\
Pooled = \rightarrow \left( \begin{matrix} 0.5 & 1 & 1.25 \end{matrix} \right) $$

In [10]:
data = [[[1.0,0.0,2.0],
         [0.0,1.0,3.0],
         [1.0,3.0,0.0],
         [0.0,0.0,0.0]]]            

x_data = torch.tensor(data)                 # 1 x InLen X EmbSize
m = nn.AvgPool1d(kernel_size=4, stride=1)

m(x_data.transpose(1, 2)).transpose(1, 2), x_data.shape

(tensor([[[0.5000, 1.0000, 1.2500]]]), torch.Size([1, 4, 3]))

## <a id='toc1_2_'></a>[Идея умного пуллинга - механизм внимания](#toc0_)

Аналогия:
- входные векторы — это люди, которые что-то обсуждают вместе. 
- вы — один из участников такого обсуждения и вы хотите высказаться. 
- вы хотите чтобы вас услышали, но все вокруг бурно дискутируют, поэтому бессмысленно что-то пытаться говорить
- сначала нужно привлечь к себе внимание, и убедиться, что вас слушают, 
- после этого можно говорить то что хотите сказать. 

Таким образом, гораздо более эффективно разделить операции **оценки значимости информации** (то есть, привлечение внимания), и **использования этой информации** (то есть, передачи дальше по нейросети).

Более формально:

1. На вход подаются вектора слов/токенов текста - матрица: 
$$ Words \in \R^{L \times S}$$
2. Для каждого вектора независимо расчитывается оценка значимости - **релевантность\***, которая ничем не ограничена  (т.е. применяется какбэ "нейросеть" с 1 выходом, у которой все веса постоянны, например однослойная свертка Conv1d, но ничего не мешает применять более сложные преобразования). Считаем, что полученные оценки это Logit (логарифм вероятности/шансов, тут у меня путаница):
$$UnnormScores = Net(Words) \in \R^L$$
3. Полученные оценки нормируются софтмаксом к [0, 1] превращаясь в "вероятности", ну или веса:
$$AttScores = SoftMax(UnnormScores) \in \R^L \\
Softmax(x) = \left\{ \frac{e^{x_i}}{\sum_j e^{x_j}}\right\}_i$$
4. Полученные вектор оценок/весов/вероятностей длинной по количеству слов в матрице характеризует важность каждого слова/токене в исходном тексте. Но его размерность не позволяет сравнивать тексты между собой, т.к. они разной длины, поэтому его нужно привести к единой размерности, которой в анализе текстов явялется размерность **эмбеддинга**. Для это этот вектор оценок слов умножается на матрицу слов:
$$Result = AttScores_{1 \times L} \cdot Words_{L \times S} \in \R^S$$

\*) Релеваантность (от англ. relevant — существенный, уместный) в информационной науке и информационном поиске означает степень соответствия найденного документа или набора документов информационным нуждам пользователя (Релевантность, Relevance).

Другими словами, мы просто взяли взвешенную сумму эмбеддингов слов данного текста, а в качестве весов взяты нормированные к единице свертки этих эмбеддингов. 

Как здесь обеспечивается привлечение внимания к конкретному слову? Наверно настройкой свертки, т.к. параметрами ее ядра, которые и определяют важность тех или иных компонентов эмбеддинга. А т.к. на выходе нам нужно ровно одно число, то окно свертки (для случая одномерной свертки) это длина эмбеддинга и все определяется ядром, также размера эмбеддинга. А эначит что это ядро - это просто тот образец эмбеддинга, по близости к которому и оценивается важность векторов слов для текста.

**Примените механизм внимания к входной матрице:**

$$Input = \left( \begin{matrix} 1 & 0 & 2 \\ 0 & 1 & 3 \\ 1 & 3 & 0 \\ 0 & 0 & 0\end{matrix} \right) \in \mathbb{R} ^ {InLen \times EmbSize}$$ 

с учётом оценок значимости $AttScores = \left( \begin{matrix} 0.1 & 0.5 & 0.3 & 0.1 \end{matrix} \right)$

Технически, основное отличие механизма внимания от avg-пулинга - наличие весов при слагаемых: 
$$Attention[Ch] = \sum_{i=0}^{InLen-1} AttScores[i] \cdot Input[i,Ch]$$

где $Input \in \mathbb{R} ^ {InLen \times EmbSize}$ - матрица признаков входной последовательности, $Ch$ - номер столбца (номер признака), $AttScores[i]$ - оценка значимости для i-го элемента входной последовательности ($AttScores \in \mathbb{R}^{InLen}, 0 \le AttScores[i] \le 1$).

In [83]:
ker = torch.tensor([0.1, 0.5, 0.3, 0.1])
ker @ x_data

tensor([[0.4000, 1.4000, 1.7000]])

## <a id='toc1_3_'></a>[Механизм внимания как набор компонентов](#toc0_)

Механизм включает следующие компоненты:

1. Входные данные (Inputs)
- матрица слов
- вектор запрос (опционально должен уметь считать)
2. Механизм расчета релевантности (UnnormScores)
- как правило это простое скалярное произведение (с ядром/образцом) или нейросеть
3. Механизм вычисления значений (AttScores)
- обычно это софтмакс (нормирование к 1)
4. Механизм получения единого вектора (агрегации)
- обычно это взвешенная сумма, т.е. тоже скалярное произведение, то с AttScores

Учитывая такую структурированность, придумана **масса вариантов** данного механизма в целом

**Преимущества механизма внимания:**
- большая гибкость по сравнению с avg/max агрегацией
- значения результирующего вектора не являются независимыми, одни компоненты входов работают на привлечение внимания, а другие на передачу информации (видимо, имеется в виду про длину (внимание) и направление (информация) вектора))
- лучше работают на длинных последовательностях (по сравнению с пуллингом и RNN)
- представляет собой универсальную операцию, которая может заменить и RNN и CNN, можно всю нейросеть построить на механизме внимания
- возможность интерпретации решений, принятых нейросетью, если добавить к ней этот механизм

## <a id='toc1_4_'></a>[Популярные архитектуры механизма внимания](#toc0_)

1. Пропущенные через софтмакс оценки релевантности (вектор внимания) на последнем этапе умножается не на входы (исходные данные), а на входы, обработанные например сверточным слоем, выделяющим более явно информацию из исходных данных.
   - т.е. входы пропускаются через 2 различные сверточные сети, одна дает оценку релевантности (внимание), которые называют "**ключи**", вторая выделяет признаки (информация), которые называют "**значения**"
2. Часто нужно оценивать релевантность слов тексте не в рамках некоторой задачи классификации, а относительно некоторого запроса извне (еще некоторого слова или набора слов, но тогда его нужно как то, агрегацией или механизмом внимания, самого привести к форме одного вектора, чтоб работало скалярное умножение)
   - в таких случаях релевантность оценивается простым скалярным произведением вектора запроса и векторов из входной матрицы. Это можно также записать как матричное произведение вектора на всю матрицу
   - иначе говоря, чем ближе векторы входных слов к вектору запроса в смысле некоторой метрики (например, косинусной), тем более они значимы
3. Можно объеденить оба этих варианта
   - это даст больше гибкости
4. И наконец, вектор запроса можно получать не извне, а из самого текста (например, глобальным пулингом)
   - оценка значимости каждого слова будет обусловлена на весь текст (мы потеряем часть пространственной информации, но увеличим часть смысловой информации)
   - технически, если это макс-пуллинг, то это получается чето типа усилителя - векторы слов увеличиваются пропорционально максимальным своим компонентам, если усреднение - то ембеддинги увеличиваются пропорционально близости к среднему

Механизмы внимания, использующие в качестве **ключей** и **значений** элементы одного и того же текста ещё называются механизмами "самовнимания" (или "self-attention"). Такое внимание уже, в принципе, может заменить рекуррентную нейросеть в ряде задач.


**Пример механима внимания**

Рассмотрим механизм внимания, в котором релевантности элементов входной последовательности расчитываются с помощью скалярного произведения с фиксированным вектором-запросом.

Как и раньше, $Input \in \mathbb{R} ^ {InLen \times EmbSize}$ - входная матрица.

Релевантности элементов входной последовательности можно посчитать с помощью матричного произведения $UnnormScores = Input \cdot Query$, где $Query \in \mathbb{R}^{EmbSize}$ - вектор-запрос, характеризующий значимость признаков. Тогда $UnnormScores \in \mathbb{R}^{InLen}$ - вектор релевантностей элементов входной последовательности (чем больше $UnnormScores[i]$, тем значимее i-ый элемент).

Затем релевантности нормируют: $AttScores = Softmax(UnnormScores)$.

Ну и наконец, результат операции внимания считается по формуле из предыдущей задачи $Attention[Ch] = \sum_{i=0}^{InLen-1} AttScores[i] \cdot Input[i,Ch]$

Примените механизм внимания к входной матрице

$$Input = \left( \begin{matrix} 1 & 0 & 2 \\ 0 & 1 & 3 \\ 1 & 3 & 0 \\ 0 & 0 & 0\end{matrix} \right)$$

с учётом вектора-запроса $Query = \left( \begin{matrix} 0 & 0 & 1 \end{matrix} \right)$.

In [84]:
import numpy as np

softmax = lambda x: np.exp(x) / np.exp(x).sum()

x_data = np.array([[1,0,2],[0,1,3],[1,3,0],[0,0,0]])
query = np.array([0,0,1])

unnorm_scores  = x_data @ query.T
att_scores = softmax(unnorm_scores)
attention = att_scores @ x_data
attention

array([0.28461991, 0.78323514, 2.54574246])

## <a id='toc1_5_'></a>[Сравнение текстов с механизмом внимания](#toc0_)

Даны 2 текста длины $L_1, L_2$. 

Тексты построены с одинм эмбеддингом, поэтому можно их матрицы перемножить. Элемент этой матрицы $L_1 \times L_2$ будет нести смысл:
- $(i, j)$ элемент показывает, насколько i-ое это слово из первого текста похоже на j-ое слово из второго текста

Есть 2 варианта — два измерения, по которым можно нормализовать (то есть, к которым можно применить софтмакс и получить $L_1 \times L_2$ "вероятностей")  
- если нормализуем по столбцам ($L_2$), т.е. умножим "вероятности" софтмакса на $L_1$, то, в результате, получим матрицу, физический смысл которой:
  - размер будет соответствовать размеру второго текста, 
  - i-ый столбец ($EmbSize \times 1$) будет хранить представление **всего** первого текста относительно i-го слова второго текста

Такие представления могут быть полезны, например, в **вопросно-ответных системах**, когда первое предложение обозначает вопрос, а второе — текст, в котором мы ищем ответ. Тогда, после такой агрегации по каждому вектору, в полученной матрице мы можем, например, предсказать, содержится ли в районе этого слова ответ на вопрос, или нет. 

Или же мы можем нормализовать матрицу релевантности по строкам — тогда получим матрицу, соответствующую размеру первого предложения, и физический смысл отобразится зеркально.

# <a id='toc2_'></a>[Трансформер и self-attention](#toc0_)

<img src="./img/transformer.png" width="500">

Трансформер и механизм внутреннего внимания (или self-attention) - модель, которая очень хорошо встряхнула всю область обработки текстов. 

Вспомним основные архитертуры НС для текста:

- RNN
  - последовательные вычисления (LSTM, GRU)
  - улучшенный параллелизм (SimleRU)
  - стоимость учета зависимости/закономерностей длины $n$ требуется $O(n)$ операций
    - при этом важные закономерности в естественном языке носят нелокальный характер
- CNN
  - параллельные вычисления ОК
  - длина учитываемых последовательностей (рецептивное поле) зависит от глубины сети и $k$
    - либо увеличивать количество слоев
    - либо увеличивать размер ядра свертки
  - **непрерывные свертки** - стоимость учета зависимости/закономерностей длины $n$ требуется $O(n/k)$ операций
  - **прореженные свертки** (dilated) - стоимость $O(\log_k(n))$ операций

Связанные с этим проблемы:
- информация о далеких зависимостях теряется
  - есть гипотеза о том, что  чем больше нам нужно сделать шагов, чтобы учесть какую-то зависимость, чтобы перенести информацию из одной точки в другую, тем выше вероятность того, что мы потеряем много информации на этом пути, и, следовательно, будем хуже учитывать далёкие зависимости
- поэтому такие нейросети либо
  -  медленно учатся, но компактнее (RNN)
  -  лучше распараллеливаются, но требуют больше вычислений (CNN)

## <a id='toc2_1_'></a>[Механизм внутреннего внимания (self-attention, intra attention)](#toc0_)

<img src="./img/self-att.png" width="700">

Разработан давно, а раскручен примерно с 2017 года гуглом. 

В целом он похож на свертки, но учитывает зависимости любой длины и вычислительно намного дешевле.

1. **Принимает на вход** матрицу токенов/признаков (ембеддинги), пусть вектора токенов - это столбцы:

$$W_{EmbSize \times Len}$$

2. **Происходит учет контекста**

$$W \rightarrow Logits_{Len \times Len} \rightarrow Scores_{Len \times Len}$$

А именно:
- в качестве векторов запроса используются все векторы  $W$, т.е. оценивается сходство токенов - каждый с каждым:
$$Logits_{Len \times Len} = W^T \cdot W$$
- с помощью софтмакса, нормируется по строкам или по столбцам (она сирамно симметичная) так, чтобы сумма весов по строке или столбцу равнялась единице (допустим по столбцам):
$$AttScores = Softmax(Logits, col)$$
- затем исходные векторы выступают в роли значений. Они взвешиваются с помощью полученных весов и складываются:
$$Result = W \cdot AttScores$$
3. **Возвращает** результирующию матрицу, где каждый токен представлен вектором признаков в контексте всех остальных токенов:
$$Result_{Len \times EmbSize} = SelfAttention = W \cdot Softmax(W^T \cdot W, col)$$

Это самый простой вариант внутреннего внимания, он демонстрирует основной принцип работы
- и нет ли тут путаницы с размерностями (W надо в конец формулы?)

### <a id='toc2_1_1_'></a>[Использование раздельных проекций для запросов, ключей и значений](#toc0_)

На практике обычно добавляется один шаг - из матрицы токенов строиться три различные проекции для пролучения векторов запросов, ключей (оценок релевантности токенов) и значений (оценок информационных признаков токенов):

$$Q = Proj_Q \cdot W \\ K = Proj_K \cdot W \\ V = Proj_V \cdot W $$

Часто используется простое линейное преобразование — домножение на матрицу. Здесь матрицы проекции — квадратные, и длина их стороны равняется размеру эмбеддинга токена.

Это даёт гораздо больше гибкости и при расчёте релевантности токен будет иметь разные признаки, в зависимости от того, выступает он в качестве запроса или в качестве ключа. 

Такое преобразование можно понимать как применение одномерной свёртки с ядром размера 1 (`Conv1d(kernel=1)`). 

Аналогично может преобразовываться и результат:

$$Output = Proj_O \cdot W $$
Итого:
$$SelfAttention = V \cdot Softmax(K^T \cdot Q, col) =\\ 
= Proj_V \cdot W \cdot Softmax(W^T \cdot Proj_K^T \cdot Proj_Q \cdot W, col)$$

**Это и есть механизм внутреннего внимания - основной строительный блок трансформера** (и многих других современных архитектур для обработки текста)

В ориг. статье это выглядит как: $A(Q,K,V) = Softmax(\frac{Q \cdot K^T}{\sqrt {d_k}})\cdot V$, где $d_k$ - размерность вектора ключей (равна $d_q$ если че). Такое нормирование, чтобы при больших размерностях в $Q \cdot K^T$ не возникало слишком больших значений, которые на софтмаксе забьют остальные - высокая контрастность.

Для одного запроса: $A(q,K,V) = \sum_i \frac{e^{q \cdot k_i}}{\sum_j e^{q \cdot k_j}})\cdot v_i$


### <a id='toc2_1_2_'></a>[Недостатки механизма внимания](#toc0_)
- **теряет информацию**: операция усреднения, пусть даже и с взвешиванием (чем является матричное произведение) - это достаточно грубая операция, много информации теряется
- **усложняет сходимость**: может концентрировать внимание только на одном аспекте сравнения:
  - т.е. внимание получит только один токен в результате единственного сравнения всех со всем единственным способом, хотя похожесть токенов может проявляться по разному


**Пример**

Механизм self-attention принимает на вход матрицу $Input \in \mathbb{R} ^ {InLen \times EmbSize}$, где $InLen$ - количество строк, соответствующее длине входной последовательности, а $EmbSize$ - количество столбцов, соответствующее количеству признаков для каждого элемента.

Найдите матрицу попарного сходства элементов $Logits=Input \cdot Input^T$.

Входная матрица имеет следующий вид

$$Input = \left( \begin{matrix} 1 & 0 \\ 0 & 1 \\ 1 & 1 \\ 0 & 0\end{matrix} \right)$$


In [85]:
x = np.array([[1, 0],[0, 1],[1, 1],[0, 0],]).T  # шоб как в формуле

logits = x.T @ x
for s in logits.tolist():
    print(*s)

1 0 1 0
0 1 1 0
1 1 2 0
0 0 0 0


**Пример**

Используя матрицу $Logits$, полученную в результате решения предыдущей задачи, найдите выходную матрицу $Result \in \mathbb{R}^{InLen \times EmbSize}$ для механизма **self-attention**.

In [86]:
att_scores = np.vectorize(softmax, signature='(d)->(d)')(logits)
result = att_scores @ x.T

for s in result.tolist():
    print(*s)
    
result.shape == x.T.shape

0.7310585786300049 0.5
0.5 0.7310585786300049
0.7310585786300049 0.7310585786300049
0.5 0.5


True

**Пример**

Посмотрим, как работает более общий вариант self-attention - когда в качестве ключей, запросов и значений используются разные матрицы. Вход тот же, что и выше. Алгоритм еще раз:

1. Найти значения ключей, запросов и значений, используя линейное преобразование
$$Keys = Input \cdot Proj_K + Bias_K \\ 
Queries = Input \cdot Proj_Q + Bias_Q \\
Values = Input \cdot Proj_V + Bias_V$$
 
2. Найти матрицу попарного сходства, используя полученные матричное произведение запросов и ключей $Logits=Queries \cdot Keys^T$
3. Найти коэффициенты усреднения, нормировав матрицу попарного сходства с помощью softmax по строкам 
$AttScores = softmax(Logits, rows)$
4. Найти результат с помощью матричного произведения матриц значений и коэффициентов $Result=AttScores \cdot Values$

вания:

Пусть:

$$Proj_K = \left(\begin{matrix}1 & 0 \\ 0 & 0\end{matrix} \right), Proj_Q = \left(\begin{matrix}0 & 0 \\ 1 & 0\end{matrix} \right), Proj_V = \left(\begin{matrix}1 & 0 \\ 0 & 1\end{matrix} \right) \\

Bias_K = Bias_Q = Bias_V = \left(\begin{matrix}0 & 0\end{matrix}\right)$$

In [87]:
proj_k = np.array([[1, 0], [0, 0]])
proj_q = np.array([[0, 0], [1, 0]])
proj_v = np.array([[1, 0], [0, 1]])

bias_k = bias_q = bias_v = np.array([[0,0]])

keys = x.T @ proj_k + bias_k
queries = x.T @ proj_q + bias_q
values = x.T @ proj_v + bias_v

logits = queries @ keys.T

att_scores = np.vectorize(softmax, signature="(d)->(d)")(logits)    # сумма по строкам == 1
result = att_scores @ values

for s in result.tolist():
    print(*s)

0.5 0.5
0.7310585786300049 0.5
0.7310585786300049 0.5
0.5 0.5


### <a id='toc2_1_3_'></a>[Повышение качества механизма внутреннего внимания. Multihead self-attention](#toc0_)

Вместо одного механизма внимания предлагается запускать несколько (Heads, "головы"), с различными весами (матрицами проекций), причем **меньшей размерности**, чем эмбеддинг исходного текста.

Таким образом Каждая "голова" работает с пространством признаков меньшего размера. Потом, на выходе, они конкатенируются — так, что размер выходной матрицы остаётся прежним — таким же, каким и был на входе. Именно такой вариант внутреннего внимания и используется в трансформере (и в большинстве современных архитектур).

Эффективный размер векторов внутри каждой "головы" - $EmbSize // HeadsN$. В целом о количестве "голов" надо думать в духе "в каком количестве разных аспектов я хочу сравнивать слова друг с другом?". Даже в больших моделях "голов" не очень много, 6-12-16.

- получается эмбеддинг токена анализируется по-частям, кусками его упорядоченных координат. Это дает то, что взвешивание релевантности токенов нормируется не по всем координатам эмбеддинга, а по частям и одни части не забивают другие (усиливаются слабые части)

В ориг.статье: 
$$head_i= Attention(QW_i^Q, KW_i^K, VW_i^V) \\
MultiHead(Q,K,V)=Concat(head_1, ..., head_h)\cdot W^O$$

<img src="./img/heads.png" width="700">


**Пример multihead self-attention**

Общий алгоритм - точно такой же, как и в предыдущей задаче. Отличие в том, что нам нужно несколько раз применить механизм внимания с разными параметрами преобразований $Result^i = SelfAttention(Input, Proj^i_K, Proj^i_Q, Proj^i_V)$, $Result^i \in \mathbb{R}^{InLen \times \frac{EmbSize} {HeadsN}}$.

Результат $MHResult \in \mathbb{R} ^ {InLen \times EmbSize}$ получается конкатенацией $Result^i$ по столбцам: $MHResult = \left[ Result^1, Result^2, ..., Result^{HeadsN} \right]$.

Вам требуется найти $MHResult$ для входных данных 

$Input = \left( \begin{matrix} 1 & 0 \\ 0 & 1 \\ 1 & 1 \\ 0 & 0\end{matrix} \right)$​
 
с учётом количества "голов" $HeadsN = 2$ и параметров преобразований

$$Proj^1_K = \left(\begin{matrix}1 & 0 \\ 0 & 0\end{matrix} \right), Proj^2_K = \left(\begin{matrix}0 & 0 \\ 1 & 0\end{matrix} \right)$$

$$Proj^1_Q = \left(\begin{matrix}0 & 1 \\ 1 & 0\end{matrix} \right), Proj^2_Q = \left(\begin{matrix}1 & 1 \\ 1 & 1\end{matrix} \right)$$

$$Proj^1_V = \left(\begin{matrix}1 \\ 0\end{matrix} \right), Proj^2_V = \left(\begin{matrix}0 \\ 1\end{matrix} \right)$$

$$Bias^i_K = Bias^i_Q = \left(\begin{matrix}0 & 0\end{matrix}\right), \quad Bias^i_V = 0$$

In [88]:
proj_k_1 = np.array([[1, 0], [0, 0]])
proj_q_1 = np.array([[0, 1], [1, 0]])
proj_v_1 = np.array([[1, 0]]).T

proj_k_2 = np.array([[0, 0], [1, 0]])
proj_q_2 = np.array([[1, 1], [1, 1]])
proj_v_2 = np.array([[0, 1]]).T

bias_k_1 = bias_k_2 = bias_q_1 = bias_q_2 = np.array([[0,0]])
bias_v_1 = bias_v_2 = np.array([[0]])

# head1
keys_1 = x.T @ proj_k_1 + bias_k_1      # 4 x 2
queries_1 = x.T @ proj_q_1 + bias_q_1   # 4 x 2
values_1 = x.T @ proj_v_1 + bias_v_1    # 4 x 1
logits_1 = queries_1 @ keys_1.T         # 4 x 4
att_scores_1 = np.vectorize(softmax, signature="(d)->(d)")(logits_1)    # 4 x 4
result_1 = att_scores_1 @ values_1      # 4 x 1

# head2
keys_2 = x.T @ proj_k_2 + bias_k_2      # 4 x 2
queries_2 = x.T @ proj_q_2 + bias_q_2   # 4 x 2
values_2 = x.T @ proj_v_2 + bias_v_2    # 4 x 1
logits_2 = queries_2 @ keys_2.T         # 4 x 4
att_scores_2 = np.vectorize(softmax, signature="(d)->(d)")(logits_2)    # 4 x 4
result_2 = att_scores_2 @ values_2      # 4 x 1

result = np.hstack((result_1, result_2))
for s in result.tolist():
    print(*s)

0.5 0.7310585786300049
0.7310585786300049 0.7310585786300049
0.7310585786300049 0.8807970779778824
0.5 0.5


## <a id='toc2_2_'></a>[Управление зависимостями](#toc0_)

Допустим, что мы делаем модель для генерации текста и хотим предсказывать следующее слово на основе предыдущих и чтобы остальные слова, которые есть на входе сети, вообще никак не учитывались. Если мы генерируем третье слово, то мы хотим чтобы учитывались только первое и второе.

В механизме внимания для работы с подобными ситуациями используется **маски**.

**Маска** — это матрица такой же размерности, что и маска "сходство токенов" ($InLen \times InLen$), элементы которой на $i$-ой/$j$-ой позиции, если для $j$-го токена надо учитывать $i$-ый токен, равные 1, если не надо учитывать - равны 0.

**Маска применяется к $Logits$, т.е. перед нормализацией матрицы сходства, т.е. перед софтмаксом.**

Тогда 

$$MaskedLogits_{ij} = \left\{ \begin{array}{ll}
      Logits_{ij}, & Masks_{ij}=1 \\
      -\inf, & Masks_{ij}=0 \\
\end{array} \right. $$

Применение софтмакса к матрице с "минус бесконечностями" приводит к тому, что на их месте появляются нули.

$$Softmax([-\inf, 1, 5]) = [0, 0.3, 0.7]$$

Механизм маскирования делает архитектуру с внутренним вниманием очень гибкой и практически универсальной. Маски можно менять динамически, для каждого слова можно формировать маску независимо от других, и так далее. Этим свойством трансформера пользуются авторы некоторых крутых архитектур, которые сейчас показывают самое лучшее качество решения многих задач.

# <a id='toc3_'></a>[Трансформер](#toc0_)

Включает несколько слоев:

1. Вход
2. Механизм внимания для учёта глобального контекста
3. Сверточные слои:
   - признаки каждого токена независимо преобразовываются с помощью двухслойной нейросети
   - ко всем токенам сеть применяется с одними и теми же параметрами
   - по смыслу это похоже на одномерные свёртки с размером ядра "1".
4. Ещё тут есть связи в обход нелинейностей, как в ResNet для классификации изображений — это ускоряет процесс обучения за счёт лучшего протекания градиентов (к выходу механиза внимания, а также к выходу сверток добавляется их выход)
5. Слои 2-4 повторяются несколько раз
6. Выход

Но, в таком виде механизм внимания ничего не знает о порядке слов!
   - RNN обрабатывают текст последовательно, такой проблемы нет
   - CNN проходят по тексту окном, и знают соседние слова в окне, такой проблемы почти нет
   - в трансформере информация о порядке слов погружается непосредственно в эмбеддинг

## <a id='toc3_1_'></a>[Позиционное кодирование](#toc0_)

1. **Кодирование позиции** (Обучаемые коды ?). Как вариант - просто складывать:
$$W(t_i) = Emb(t_i) + PosCode(t_i), \\
PosCode(t_i) \in \R^{EmbSize}$$

- часто это не вариант, т.к. кодируется только фиксированное количество позиций (например, сеть обучалась на примерах длины не более 10, она не поймет никаких других позиций), 

2. **Периодический (синусоидный) сигнал**
- подмешивать (прибавлять) его к эмбеддингу, так сеть будет узнавать/запоминать положение данного токена в тексте в рамках некоторого цикла. Какая-то такая штука (вектор гармоник какой-то):

$$PosCode(t_i) = \left[\begin{matrix} f(t_i)^{(1)} \\ f(t_i)^{(1)}  \\ ... \\ f(t_i)^{(k)} \\ f(t_i)^{(k)} \\... \\ f(t_i)^{(d/2)} \\ f(t_i)^{(d/2)} \end{matrix} \right]_{d \times 1} = \left[\begin{matrix} sin(\omega_1 \cdot i) \\ cos(\omega_1 \cdot i)  \\ ... \\ sin(\omega_k \cdot i) \\ cos(\omega_k \cdot i) \\... \\ sin(\omega_{d/2} \cdot i) \\ cos(\omega_{d/2} \cdot i) \end{matrix} \right]_{d \times 1} $$

где
$$f(t_i)^{(k)} = \left\{ \begin{array}{ll}
      sin(\omega_k \cdot i), & i = 2k \\
      cos(\omega_k \cdot i), & i = 2k+1\\
\end{array} \right. \\
\omega_k = \frac{1}{10000^{2k/d}}$$

- $d$ - размерность эмбеддинга, 
- через $k$ мы определяем значение частоты и начальной фазы (т.е. синус или косинус) для $i$-того элемента в векторе $PosCode(t_i)$, 
- $i$ - это номер токена.

В семинаре - пример, на примере - все просто.

На практике эти два подхода — обучаемые коды и периодически сигнал — работают примерно одинаково, но сигнал гораздо удобнее использовать. Вот так выглядит общая архитектура трансформера, как её нарисовали авторы. Они изначально решали задачу машинного перевода, то есть генерации предложения по предложению на другом языке. 

Поэтому их архитектура состоит из энкодера (мы его уже рассматривали) и декодера. Декодер отличается тем, что в нём используются маски. А ещё, в него подаются выходы энкодера. Но, в остальном, это практически одна и та же нейросеть (но параметры, конечно же, у них разные). И в энкодере, и в декодере, используется много слоёв вот такого типа — в оригинальной статье по 6, а потом стали делать ещё больше — по 12, по 24... 

Такая архитектура позволяет решать задачи, связанные с анализом последовательностей (не только с текстами), часто — лучше, чем это получается делать с помощью рекурренток, и, при этом, можно тратить меньше времени на обучение.

[1] [Attention and its Different Forms](https://towardsdatascience.com/attention-and-its-different-forms-7fc3674d14dc)  
[2] [Transformer Architecture: The Positional Encoding](https://kazemnejad.com/blog/transformer_architecture_positional_encoding/)

Семинар в директории `./stepik-dl-nlp`

# <a id='toc4_'></a>[Теоретические вопросы: Модель языка и трансформеры](#toc0_)

Напишите функцию, реализующую max-pooling с заданной шириной окна. В качестве пособия можно использовать другую задачу.

Функция должна возвращать два тензора одинаковой размерности $OutLen \times EmbSize$:

- первый тензор - основной результат - результат применения max-пулинга к каждому столбцу для каждой позиции скользящего окна
- второй тензор - информация, нужная для расчёта градиента, - относительные индексы максимальных элементов для каждого столбца для каждой позиции скользящего окна


In [89]:
SAMPLES = """[[0.6046018907385543, 0.0812964077275945, 0.6366439552273822, 0.20134327995534496], [0.8106187774962709, 0.4395507898340978, 0.8290096270213004, 0.05773841312522798], [0.938964520620817, 0.3860407857274528, 0.21318174478828456, 0.07860176987690048], [0.04840110723428537, 0.6411287103553837, 0.509253427569025, 0.7094441369109541], [0.691552879511939, 0.4979735285634219, 0.07060470682483455, 0.7631262538014161]]\n2""", 
READER = (x for x in SAMPLES[0].split('\n')); 
sys.stdin.readline = lambda: next(READER)       # тупо шоп не переписывать

import sys
import ast
import numpy as np


def parse_array(s):
    return np.array(ast.literal_eval(s))

def read_array():
    return parse_array(sys.stdin.readline())

def write_array(arr):
    print(repr(arr.tolist()))


def max_pooling(features, kernel_size):
    """
    features - InLen x EmbSize - features of elements of input sequence
    kernel_size - positive integer - size of sliding window

    returns tuple of two matrices of shape OutLen x EmbSize:
         - output features (main result)
         - relative indices of maximum elements for each position of sliding window
    """
    InLen = features.shape[0]
    KernelSize = kernel_size
    OutLen = InLen - KernelSize + 1

    sliding_indexer = np.expand_dims(np.arange(KernelSize), 0) + \
                      np.expand_dims(np.arange(OutLen), 0).T   # KernelSize x OutLen

    features_slide = features[sliding_indexer]
    max_pool = np.max(features_slide, axis=1)
    indicies = np.argmax(features_slide, axis=1)
    
    # обратная операция - если в argmax не доступен keepdims (numpy<1.22.0)
    # np.take_along_axis(features_slide, np.expand_dims(indicies, axis=1), axis=1)  

    return max_pool, indicies


features = read_array()
kernel_size = int(sys.stdin.readline())

result, indices = max_pooling(features, kernel_size)

write_array(result)
write_array(indices)

[[0.8106187774962709, 0.4395507898340978, 0.8290096270213004, 0.20134327995534496], [0.938964520620817, 0.4395507898340978, 0.8290096270213004, 0.07860176987690048], [0.938964520620817, 0.6411287103553837, 0.509253427569025, 0.7094441369109541], [0.691552879511939, 0.6411287103553837, 0.509253427569025, 0.7631262538014161]]
[[1, 1, 1, 0], [1, 0, 0, 1], [0, 1, 1, 1], [1, 0, 0, 1]]


Прямому проходу по модулю max-пулинга была посвящена предыдущая задача.

Теперь займёмся обратным проходом: напишите функцию, вычисляющую производную функции потерь по входам модуля max-пулинга $\frac{\partial Loss}{\partial features}$

Функция принимает следующие аргументы:

- $features \in \mathbb{R}^{InLen \times EmbSize}$ - признаки, которые были переданы на вход модулю при прямом проходе
- $2 \leq kernel\_size \leq InLen$ - размер скользящего окна
- $indices \in \mathbb{N}^{OutLen \times EmbSize}, \quad 0 \leq indices < kernel\_size$ - относительная позиция максимального элемента внутри скользящего окна для каждого элемента выходного тензора (смещение относительно номера строки)
- $dldout = \frac{\partial Loss} {\partial out} \in \mathbb{R}^{OutLen \times EmbSize}$ - значение производной функции потерь по выходам слоя max-пулинга (то есть производная по входам следующего слоя)

Вам может быть полезна формула производной кусочно-линейной функции 

$$ReLU(x) = \begin{cases} 0 & if \quad x \leq 0 \\ x & otherwise \end{cases} \\
 \frac{\partial ReLU(x)} {\partial x} = \begin{cases} 0 & if \quad x \leq 0 \\ 1 & otherwise \end{cases}$$ 

А также помните про правило цепочки: $\frac{\partial Loss} {\partial features} = \frac{\partial Loss} {\partial out} \frac{\partial out} {\partial features}$.

**Получилось не очень...**

In [205]:
SAMPLES = """[[-1.2420542766989977, -0.045100789663994285, 1.858151857421511, 0.10732741246325356], [-1.480497780371414, -0.12486054931133332, -0.18422425981847368, -1.4228130362490647], [-0.8417536968892625, 0.9802583655274091, -0.18413492661665792, -1.5582607186399924], [1.325799250424393, 0.08149768959330334, -1.454876921308986, 0.1408031456023352], [0.1637602967235608, -0.21250114632967532, 0.8362859721448469, 0.717774697701287], [-0.7641399532198978, -2.112568530488304, 0.20121440705964902, 0.015624280892385661], [1.3862200103422582, 0.6508694196448389, -1.162417318743681, 1.5202488401790915], [1.3947418297193952, -1.013483406336198, -2.0608332074129545, -1.733019236247151], [1.0932612618870112, 0.8071262618398916, 0.15924519176972282, -0.6885825807454318]]\n6\n[[3.0, 2.0, 0.0, 4.0], [5.0, 1.0, 3.0, 5.0], [5.0, 0.0, 2.0, 4.0], [4.0, 5.0, 1.0, 3.0]]\n[[-0.0763791951131031, -0.8729161683329371, 0.7454337173675266, -2.508470377801969], [-0.9080189656976042, 0.6952579391985969, 0.2829942797947518, 0.35396918585149195], [0.339009358836277, -1.9496823733556254, -0.11017174942549533, 1.4591363247582954], [-1.1161622731011638, -2.371055190136431, -1.174384738333498, 0.4672192180206214]]""", \
          """[[-4.15901891e-02, -1.56756601e+00, -1.23642116e+00],[-1.50168154e-02,  2.62078972e-02, -9.12503559e-01],  [-8.97504443e-01,  4.80590968e-01, -1.53892657e+00],  [-7.95696114e-01,  8.81377550e-04, -7.66638264e-02],  [-1.38615994e+00, -1.58366520e+00, -5.78961762e-01],  [ 4.34240133e-01,  9.26438834e-01,  7.38135813e-01]]\n5\n[[1, 2, 3],[4, 4, 4]]\n[[-1.13774517, -1.3217654 , -0.45930326],  [ 0.05638115,  0.77569495, -0.02360533]]""", 
READER = (x for x in SAMPLES[0].split('\n')); 
sys.stdin.readline = lambda: next(READER)  # тупо шоп не переписывать


def max_pooling_dldfeatures_fast(features, kernel_size, indices, dldout):
    """Хорошее решение"""
    doutdfeatures = np.zeros(features.shape)
    out_len = indices.shape[0] 
    for i in range(kernel_size):
        doutdfeatures[i:i+out_len] += (indices == i) * dldout
    return doutdfeatures

def max_pooling_dldfeatures_fasta(features, kernel_size, indices, dldout):
    result = np.zeros(features.shape)
    for i in range(features.shape[0] - kernel_size + 1):
        result[indices[i, :]+i, range(features.shape[1])] += dldout[i,:]
    return result

def max_pooling_dldfeatures(features, kernel_size, indices, dldout):
    """
    features - InLen x EmbSize - features of elements of input sequence
    kernel_size - positive integer - size of sliding window
    indices - OutLen x EmbSize - relative indices of maximum elements for each window position
    dldout - OutLen x EmbSize - partial derivative of loss function with respect to outputs of max_pooling layer

    returns InLen x EmbSize
    """
    in_len = features.shape[0]
    out_channels = indices.shape[0]
    
    # None в индексах добавляет пустую размерность, нужно для броадкаста 
    abs_indices = indices + np.arange(out_channels)[None,:].T
    loc_indices = np.zeros_like(features) + np.arange(in_len)[None,:].T
    mask = loc_indices == abs_indices[:, None] 
    dldf_by_out_channel = mask * dldout[:, None]

    return np.sum(dldf_by_out_channel, axis=0)


features = read_array()
kernel_size = int(sys.stdin.readline())
indices = read_array().astype('uint32')
dldout = read_array()

dldfeatures = max_pooling_dldfeatures(features, kernel_size, indices, dldout)

write_array(dldfeatures)

[[0.0, 0.0, 0.7454337173675266, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, -2.1273406024899657, 0.0, 0.0], [-0.0763791951131031, 0.0, 0.0, 0.0], [0.0, 0.0, -1.0015622079642417, -2.508470377801969], [0.0, 0.0, 0.0, 0.0], [-0.9080189656976042, 0.0, 0.0, 2.2803247286304087], [-0.7771529142648868, 0.0, 0.0, 0.0], [0.0, -2.371055190136431, 0.0, 0.0]]


In [206]:
scale = 100

# avoid caching
x = np.random.random_sample([scale*s for s in features.shape])
ker_size = kernel_size * scale // 2
dldo, ind = max_pooling(x, ker_size)

ker_size, [i.shape for i in (x, ind, dldo)]

(300, [(900, 400), (601, 400), (601, 400)])

In [207]:
%%time
tmp0 = max_pooling_dldfeatures_fast(x, ker_size, ind, dldo)

CPU times: user 418 ms, sys: 5.61 ms, total: 423 ms
Wall time: 440 ms


In [208]:
%%time
tmp1 = max_pooling_dldfeatures_fasta(x, ker_size, ind, dldo)

CPU times: user 49.8 ms, sys: 2.41 ms, total: 52.2 ms
Wall time: 68.1 ms


In [209]:
%%time
tmp2 = max_pooling_dldfeatures(x, ker_size, ind, dldo)

CPU times: user 1.16 s, sys: 297 ms, total: 1.46 s
Wall time: 1.52 s


In [95]:
np.allclose(tmp1, tmp0)

True

Вектор-функция $softmax(x) = \left( \begin{matrix} \frac{e^{x_1}}{\sum_j e^{x_j}} & ... & \frac{e^{x_n}}{\sum_j e^{x_j}} \end{matrix} \right)$ - популярный способ нормировать вектор чисел $x \in \mathbb{R}^n$ так, чтобы $0 \leq softmax_i(x) \leq 1$ и $\sum_i softmax_i(x) = 1$.

Напишите функцию, вычисляющую softmax для заданного вектора.

In [96]:
SAMPLES = """[0.7903253367110061, 0.1738679257213426, 0.6840758121402977, -1.4921922753864911, -0.20701526564877176, -0.13343908330179777, 0.27078275189785883, -0.47987385916752834, 1.762361457920409, -0.6382574781276095, -1.476682298043406, 0.13308403857533435, 0.08629164346752129, -0.15274120983311792, 0.03422761142701722, 2.0977915122558075, -0.09695813983037735, 1.145554587286743]""",
READER = (x for x in SAMPLES[0].split('\n')); 
sys.stdin.readline = lambda: next(READER)  # тупо шоп не переписывать


def softmax(x):
    """
    x - vector of n elements - input

    returns vector of n elements - softmax output
    """
    return np.exp(x) / np.exp(x).sum()


x = read_array()

result = softmax(x)

write_array(result)

[0.06860598204389033, 0.03703718180652343, 0.06169051603553571, 0.006999663809155501, 0.02530593948353462, 0.02723806143745914, 0.040806327204274295, 0.019262891730251305, 0.18134764032840614, 0.01644130749673833, 0.007109074723334883, 0.03555704949423753, 0.033931576448048006, 0.0267173505099923, 0.032210162471156406, 0.25362223807297396, 0.028250079060620836, 0.09786695784386741]


Напишите функцию, реализующую производную выхода softmax по входам $\frac{\partial softmax(x)} {\partial x}$

Помните, что и $softmax$ и $x$ - вектора из $n$ элементов, поэтому производная - это матрица, в $ij$-ячейке которой стоит производная $i$-го элемента $softmax_i(x)$ по $j$-му входу $x_j$ (i соответствует номеру строки, j - номер столбца). Такая матрица ещё называется матрицей Якоби.

Подсказка: возможно, будет проще решать эту задачу, рассматривая два случая, когда $i=j$ и когда $i \neq j$.

In [97]:
SAMPLES = """[-0.36170314084137395, 1.531638983431016, -1.7131284538840788, -0.9027503682845508, -0.8591376176087115, 0.16481576122888014, 0.2286590883015934, 0.4686776665093307, -0.4880318948728026, 0.13865483857501165, -1.0740873577508447, -1.1693607815929845, 0.6388250392697977]""",
READER = (x for x in SAMPLES[0].split('\n')); 
sys.stdin.readline = lambda: next(READER)  # тупо шоп не переписывать

def dsoftmax_dx(x):
    """
    x - vector of n elements - input

    returns matrix n x n
    """
    s = softmax(x)
    jacobian_m = np.diag(s)

    for i in range(len(jacobian_m)):
        for j in range(len(jacobian_m)):
            if i == j:
                jacobian_m[i][j] = s[i] * (1 - s[i])
            else: 
                jacobian_m[i][j] = -s[i] * s[j]
    return jacobian_m

def dsoftmax_dx_vect(x):
    s = softmax(x).reshape(-1,1)
    return np.diagflat(s) - np.dot(s, s.T)


x = read_array()

result = dsoftmax_dx(x)

write_array(result)

[[0.04520990642005896, -0.01496136384662353, -0.0005831584750616378, -0.0013113823146393676, -0.001369840806658998, -0.0038138833133273915, -0.004065315035397694, -0.005168124298133091, -0.001985359996840069, -0.0037154023993240712, -0.0011048889071803723, -0.0010044813810742047, -0.006126705645798534], [-0.01496136384662353, 0.21585791972231008, -0.0038730635991859903, -0.008709582942971316, -0.009097836680483412, -0.025330014505627975, -0.026999905439354373, -0.03432424452555927, -0.0131858248895869, -0.024675950714133716, -0.007338151103938566, -0.006671291663363668, -0.04069068981148137], [-0.0005831584750616378, -0.0038730635991859903, 0.012135730472354802, -0.0003394788843793973, -0.0003546120941472777, -0.000987303883778268, -0.0010523922714690004, -0.0013378776360475468, -0.0005139521780146152, -0.0009618100285956132, -0.0002860237242683219, -0.000260031125035206, -0.0015860265723719278], [-0.0013113823146393676, -0.008709582942971316, -0.0003394788843793973, 0.0268663945909074

Напишите функцию, реализующую простой механизм внимания.

In [98]:
SAMPLES = """[[0.3504305689198156, 0.871844425624726, 0.29345316540775357, 0.49159320438393916, 0.16391992930609034, 0.24589641847050037, 0.34921020303336925, 0.09968814035867879, 0.8652385667745919, 0.00906484385602968, 0.6134586521086117, 0.08104312584086149, 0.643129733435556, 0.6610968673257929, 0.6825169003800382], [0.005641686042561211, 0.3397733866278605, 0.4408793722092307, 0.6618752692611525, 0.4192615374283991, 0.6718589897811911, 0.23503584107912667, 0.9972834040264165, 0.6907780153811639, 0.5160598448726361, 0.4200243418824855, 0.7745997472321381, 0.9124177261957108, 0.627661131744206, 0.7792319239076758], [0.44947044223343224, 0.39851993627332394, 0.27645205987950927, 0.3360502940952873, 0.20207394761469466, 0.27730469648938627, 0.9647449489128369, 0.38480306917172535, 0.7014748335636187, 0.5616919724157547, 0.3082991954077743, 0.43320540280287834, 0.7682716834674514, 0.04669826413239708, 0.7975639937877288], [0.3529089999231677, 0.5085801437940869, 0.6686697864089949, 0.9579051761714787, 0.14893344048972235, 0.6504801381978066, 0.6554087909852483, 0.2182290972559655, 0.8874805349214916, 0.8549111586624765, 0.2256959432511686, 0.7090739886051667, 0.9215898392742404, 0.8361038069049278, 0.9807575571901945], [0.93096165894251, 0.21204415175966806, 0.005393301311816812, 0.6868163541395496, 0.17651743121260177, 0.4276211882347165, 0.7172630747046246, 0.6222321154458413, 0.782866044726867, 0.9401403417712956, 0.6009251310321828, 0.7689712777389628, 0.011137370287858661, 0.6750220130270511, 0.3656918897133191], [0.344423832576648, 0.5200078573131781, 0.08090528060543856, 0.6187002344784092, 0.24428489996011238, 0.18400539459399, 0.40308101020726217, 0.19255989698913867, 0.8010944590934469, 0.20324438899818598, 0.4927144298170735, 0.03783988662477278, 0.7705093963091103, 0.2520865403496373, 0.40266725180440743], [0.6681310493060861, 0.2801674372250399, 0.6224648405839522, 0.6287746784150413, 0.864080498689899, 0.23833127705610258, 0.20311743810136906, 0.6646132937899738, 0.23575457417289924, 0.1869625695994146, 0.7712148738157554, 0.15237041670323637, 0.2763150373902683, 0.46500408886101585, 0.991468614310106], [0.47955269815875845, 0.18371674117676162, 0.4749895427072034, 0.5127159626377625, 0.14327300286458633, 0.5921086963579639, 0.21467664382766927, 0.08984875049424312, 0.5619088573772313, 0.6324525220346037, 0.65145500723789, 0.5118736033583858, 0.3791794826772541, 0.7062193547285907, 0.12888775429739185], [0.8936198110390413, 0.1499351596848777, 0.23230300209801535, 0.6275970485906217, 0.14179412142521963, 0.44590423506527455, 0.6398118989481705, 0.44025473834142315, 0.9690917160909921, 0.7329911430579539, 0.4723409208689966, 0.30051845327308735, 0.6517065287372249, 0.11682964074366375, 0.3356912564688531], [0.5819483213886298, 0.12715730125007862, 0.6624081011547434, 0.27210122174739293, 0.3321414469174103, 0.6738692045564213, 0.49979407643990537, 0.923095453187033, 0.8688108133354637, 0.29672803800781, 0.8422355709858387, 0.22967466587429908, 0.48606633908371566, 0.3302629931498261, 0.9271715208940098], [0.814229644181095, 0.6269508015754929, 0.19067116118180638, 0.6597416333912787, 0.3042396495694798, 0.5349586078017191, 0.9889297007928726, 0.5059647631701082, 0.6586214714303461, 0.19972197385065704, 0.730120041302739, 0.9254129585548022, 0.7774768791337286, 0.5880525770183761, 0.4404909426586451], [0.8393608070151912, 0.551470751477307, 0.3776646281929925, 0.7403545806778788, 0.01464073752506101, 0.49682079457661743, 0.12829037985166736, 0.8323709714882789, 0.4861583628986299, 0.10966510942571872, 0.36384711262095637, 0.008343156485128067, 0.05481969871494197, 0.11036480291979456, 0.3495717657917], [0.575668909069271, 0.1948209406820347, 0.5066632418120769, 0.5610866065811511, 0.7503051258152065, 0.20250301475454058, 0.9387177222181186, 0.4214964558859865, 0.2441688535705906, 0.2852282954051667, 0.7185375048873539, 0.09961745251862686, 0.507873295740294, 0.9713796833363287, 0.8218946227484244], [0.14519733396011691, 0.07264015089790021, 0.7254237309701331, 0.7437297525624584, 0.3465971472185204, 0.6489212261982703, 0.2152569561085349, 0.6476151760429897, 0.2045187871395916, 0.9599712380254137, 0.28554199184758966, 0.7701922251424572, 0.7095119328780166, 0.7579558453415812, 0.4251898428876446]]\n[0.30760147020407946, 0.1528992448227442, 0.9387231083505163, 0.12201982125460176, 0.3159744925438269, 0.555332538642272, 0.8654043562058316, 0.5523485724329922, 0.6405492495162189, 0.8421217300945876, 0.03415012932624606, 0.0914780538557024, 0.745151636966557, 0.9885343010237021, 0.02289480154711454]""",
READER = (x for x in SAMPLES[0].split('\n')); 
sys.stdin.readline = lambda: next(READER)  # тупо шоп не переписывать

def attention(features, query):
    """
    features - InLen x EmbSize - features of elements of input sequence
    query - EmbSize - features of query object

    returns vector of size EmbSize - features, aggregated according to the query
    """
    unnorm_scores  = features @ query.T
    att_scores = softmax(unnorm_scores)
    attention = att_scores @ features
    attention

    return attention


features = read_array()
query = read_array()

result = attention(features, query)

write_array(result)

[0.4740581778159266, 0.34432834551260527, 0.4665167530554428, 0.6711507992179184, 0.3014932578908269, 0.5216013512810533, 0.5739206841874825, 0.5011664230502015, 0.6659426824711602, 0.5870403517141657, 0.4845604544566492, 0.5519092842133653, 0.6631513043916593, 0.6203158151313123, 0.679961421985706]


Напишите функцию, реализующую механизм self-attention внимания с линейными преобразованиями

In [99]:
SAMPLES = """[[0.5175129778200084, 0.13021330700949507], [-0.3578609445921744, -0.07768163060380659], [-0.046577477754636734, -0.12288550821838619], [0.4424092505449793, -1.431399548551344], [0.753992222548331, -1.1210257338970167], [1.6736061037504428, -1.9789731491226337], [-1.4985152255486565, -1.6614802556117283], [-0.610065708073959, -0.8475335063027695], [-0.1657640783522184, -1.7079776825852762], [0.7857341373616981, 0.2956255012408635], [-0.49243028413686984, 0.01065675311085114], [0.20598401523943788, -0.6339670563637549], [-0.15698934123474126, 0.9516567843056503], [-0.08965595798444444, 0.9923765422389716], [-0.9649404480809814, -0.6203623955866846]]\n[[-0.4777373377067222, 1.384780896738418], [1.5537173245233542, -1.6151073640132454]]\n[0.34068677065672126, -1.7225350645946451]\n[[1.667192089239799, 0.9072106203091014], [1.0017863742909645, -0.8578876449703756]]\n[-0.3811843050970959, -0.15922065479353645]\n[[2.396375885002737, 0.23995979796695774], [0.21631803999882093, -0.6475173781192963]]\n[1.448781271879823, -0.7144164516316529]""",
READER = (x for x in SAMPLES[0].split('\n')); 
sys.stdin.readline = lambda: next(READER)  # тупо шоп не переписывать


def self_attention(features, proj_k, bias_k, proj_q, bias_q, proj_v, bias_v):
    """
    features - InLen x EmbSize - features of elements of input sequence
    proj_k - EmbSize x EmbSize - projection matrix to make keys from features
    bias_k - EmbSize - bias vector to make keys from features
    proj_q - EmbSize x EmbSize - projection matrix to make queries from features
    bias_q - EmbSize - bias vector to make queries from features
    proj_v - EmbSize x EmbSize - projection matrix to make values from features
    bias_v - EmbSize - bias vector to make values from features

    returns InLen x EmbSize
    """
    # head1
    keys = features @ proj_k + bias_k     
    queries = features @ proj_q + bias_q  
    values = features @ proj_v + bias_v   
    logits = queries @ keys.T        
    att_scores = np.vectorize(softmax, signature="(d)->(d)")(logits)   
    result = att_scores @ values     
        
    return result


features = read_array()
proj_k = read_array()
bias_k = read_array()
proj_q = read_array()
bias_q = read_array()
proj_v = read_array()
bias_v = read_array()

result = self_attention(features, proj_k, bias_k, proj_q, bias_q, proj_v, bias_v)

write_array(result)

[[1.3828516277455327, -0.7052491362196253], [1.2990231347325172, 0.07313412990950194], [1.7291639000678214, 0.057097555962845575], [4.9977493686508465, 0.9611194260544595], [4.920776273177979, 0.9417305419570732], [5.027240226701748, 0.9675417617162319], [5.0084058482460545, 0.9647146788718596], [4.6177066707952, 0.8915957294670734], [5.01858654091172, 0.966041694502124], [1.412170986514584, -1.0082854213057408], [0.8082085153678324, -0.028634744390539153], [4.261630070405553, 0.7878600291134994], [1.1488305812076605, -1.188280258100271], [1.1929080669359462, -1.220148314614849], [3.906830387768465, 0.7744398548226678]]
